### Homework 07

Name: Yining Liu  
Github Username: Lynzz1701    
USC ID: 6168529797 

In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing, metrics
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, cross_validate
from sklearn.svm import SVC, LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.cluster import KMeans

from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as imbpipeline

import os

import matplotlib.pyplot as plt
import seaborn as sns

## 1. Multi-class and Multi-Label Classification Using Support Vector Machines
#### 1(a) Choose 70% of the data randomly as the training set.

In [2]:
MFCC = pd.read_csv('../data/AnuranCalls/Frogs_MFCCs.csv')
MFCC

,MFCCs_ 1,MFCCs_ 2,MFCCs_ 3,MFCCs_ 4,MFCCs_ 5,MFCCs_ 6,MFCCs_ 7,MFCCs_ 8,MFCCs_ 9,MFCCs_10,...,MFCCs_17,MFCCs_18,MFCCs_19,MFCCs_20,MFCCs_21,MFCCs_22,Family,Genus,Species,RecordID
0,1.0,0.152936,-0.105586,0.200722,0.317201,0.260764,0.100945,-0.150063,-0.171128,0.124676,...,-0.108351,-0.077623,-0.009568,0.057684,0.118680,0.014038,Leptodactylidae,Adenomera,AdenomeraAndre,1
1,1.0,0.171534,-0.098975,0.268425,0.338672,0.268353,0.060835,-0.222475,-0.207693,0.170883,...,-0.090974,-0.056510,-0.035303,0.020140,0.082263,0.029056,Leptodactylidae,Adenomera,AdenomeraAndre,1
2,1.0,0.152317,-0.082973,0.287128,0.276014,0.189867,0.008714,-0.242234,-0.219153,0.232538,...,-0.050691,-0.023590,-0.066722,-0.025083,0.099108,0.077162,Leptodactylidae,Adenomera,AdenomeraAndre,1
3,1.0,0.224392,0.118985,0.329432,0.372088,0.361005,0.015501,-0.194347,-0.098181,0.270375,...,-0.136009,-0.177037,-0.130498,-0.054766,-0.018691,0.023954,Leptodactylidae,Adenomera,AdenomeraAndre,1
4,1.0,0.087817,-0.068345,0.306967,0.330923,0.249144,0.006884,-0.265423,-0.172700,0.266434,...,-0.048885,-0.053074,-0.088550,-0.031346,0.108610,0.079244,Leptodactylidae,Adenomera,AdenomeraAndre,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7190,1.0,-0.554504,-0.337717,0.035533,0.034511,0.443451,0.093889,-0.100753,0.037087,0.081075,...,0.069430,0.071001,0.021591,0.052449,-0.021860,-0.079860,Hylidae,Scinax,ScinaxRuber,60
7191,1.0,-0.517273,-0.370574,0.030673,0.068097,0.402890,0.096628,-0.116460,0.063727,0.089034,...,0.061127,0.068978,0.017745,0.046461,-0.015418,-0.101892,Hylidae,Scinax,ScinaxRuber,60
7192,1.0,-0.582557,-0.343237,0.029468,0.064179,0.385596,0.114905,-0.103317,0.070370,0.081317,...,0.082474,0.077771,-0.009688,0.027834,-0.000531,-0.080425,Hylidae,Scinax,ScinaxRuber,60
7193,1.0,-0.519497,-0.307553,-0.004922,0.072865,0.377131,0.086866,-0.115799,0.056979,0.089316,...,0.051796,0.069073,0.017963,0.041803,-0.027911,-0.096895,Hylidae,Scinax,ScinaxRuber,60


In [3]:
X = MFCC.iloc[:,:22]
y = MFCC.iloc[:,22:25]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=13)
y1_train, y2_train, y3_train = y_train['Family'], y_train['Genus'], y_train['Species']
y1_test, y2_test, y3_test = y_test['Family'], y_test['Genus'], y_test['Species']

#### 1(b)i Research exact match and hamming score/loss methods for evaluating multi-label classification and use them in evaluating the classifiers in this problem.

In this problem 3 separate SVMs are trained and preds are made for each label. After combining preds into tuples, we could calculate exact match ratios and hamming scores/losses by their definition.

#### 1(b)ii Train a SVM for each of the labels, using Gaussian kernels and one versus all classifiers. Determine the weight of the SVM penalty and the width of the Gaussian Kernel using 10 fold cross validation. 

In [4]:
#SVC-Gaussian-Family(y1)
parameters = {'gamma':np.arange(0.1,4.1,0.1), 'C':[0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000, 100000]}
svm = SVC(kernel='rbf',decision_function_shape='ovr')
clf = GridSearchCV(svm, parameters, scoring='accuracy', cv=10, n_jobs=-1)
clf.fit(X_train, y1_train)
#clf.cv_results_
best_svm_Family = clf.best_estimator_

print("Best estimator for Family:", clf.best_estimator_)
print("With a CV score of ", clf.best_score_)
print("Train score for this clf = ", best_svm_Family.score(X_train, y1_train))

Best estimator for Family: SVC(C=100, gamma=2.9000000000000004)
With a CV score of  0.9944424721512197
Train score for this clf =  1.0


In [5]:
#SVC-Gaussian-Genus(y2)
parameters = {'gamma':np.arange(0.1,4.1,0.1), 'C':[0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000, 100000]}
svm = SVC(kernel='rbf',decision_function_shape='ovr')
clf = GridSearchCV(svm, parameters, scoring='accuracy', cv=10, n_jobs=-1)
clf.fit(X_train, y2_train)
#clf.cv_results_
best_svm_Genus = clf.best_estimator_

print("Best estimator for Genus:", clf.best_estimator_)
print("With a CV score of ", clf.best_score_)
print("Train score for this clf = ", best_svm_Genus.score(X_train, y2_train))

Best estimator for Genus: SVC(C=10, gamma=1.3000000000000003)
With a CV score of  0.9918615292372749
Train score for this clf =  0.9994042891183479


In [6]:
#SVC-Gaussian-Species(y3)
parameters = {'gamma':np.arange(0.1,4.1,0.1), 'C':[0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000, 100000]}
svm = SVC(kernel='rbf',decision_function_shape='ovr')
clf = GridSearchCV(svm, parameters, scoring='accuracy', cv=10, n_jobs=-1)
clf.fit(X_train, y3_train)
#clf.cv_results_
best_svm_Species = clf.best_estimator_

print("Best estimator for Species:", clf.best_estimator_)
print("With a CV score of ", clf.best_score_)
print("Train score for this clf = ", best_svm_Species.score(X_train, y3_train))

Best estimator for Species: SVC(C=10, gamma=1.4000000000000001)
With a CV score of  0.9914647038404494
Train score for this clf =  0.9994042891183479


In [28]:
#exact match and hamming score for combined results: Gaussian + raw Features
preds_Family = best_svm_Family.predict(X_test)
preds_Genus = best_svm_Genus.predict(X_test)
preds_Species = best_svm_Species.predict(X_test)

print("w Gaussian + raw Features")
num_exact_match = 0
for i in range(y1_test.shape[0]):
    if((preds_Family[i]==y1_test.iloc[i])
       and(preds_Genus[i]==y2_test.iloc[i])
       and(preds_Species[i]==y3_test.iloc[i])):
        num_exact_match = num_exact_match + 1
exact_match_ratio = num_exact_match/y1_test.shape[0]
print("Exact match ratio = ", exact_match_ratio)

num_hamming_loss = 0
for i in range(y1_test.shape[0]):
    if(preds_Family[i]!=y1_test.iloc[i]):
        num_hamming_loss = num_hamming_loss + 1
    if(preds_Genus[i]!=y2_test.iloc[i]):
        num_hamming_loss = num_hamming_loss + 1
    if(preds_Species[i]!=y3_test.iloc[i]):
        num_hamming_loss = num_hamming_loss + 1
hamming_loss = num_hamming_loss/(y1_test.shape[0]*3)
print("hamming score = ", 1 - hamming_loss)
    

w Gaussian + raw Features
Exact match ratio =  0.984251968503937
hamming score =  0.9888837424733673


#### You are welcome to try to solve the problem with both standardized and raw attributes and report the results.

In [8]:
scaler = preprocessing.StandardScaler()
X_train_standardized = scaler.fit_transform(X_train)
X_test_standardized = scaler.transform(X_test)

In [50]:
#SVC-Gaussian-Family(y1):standardized
parameters = {'gamma':np.arange(0.1,3.1,0.1), 'C':[0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000]}
svm = SVC(kernel='rbf',decision_function_shape='ovr')
clf = GridSearchCV(svm, parameters, scoring='accuracy', cv=10, n_jobs=-1)
clf.fit(X_train_standardized, y1_train)
#clf.cv_results_
best_svm_Family_s = clf.best_estimator_
print("(Standardized Features)")
print("Best estimator for Family:", clf.best_estimator_)
print("With a CV score of ", clf.best_score_)
print("Train score for this clf = ", best_svm_Family_s.score(X_train_standardized, y1_train))

(Standardized Features)
Best estimator for Family: SVC(C=10, gamma=0.1)
With a CV score of  0.9904694846792262
Train score for this clf =  1.0


In [47]:
#SVC-Gaussian-Genus(y2):standardized
parameters = {'gamma':np.arange(0.1,3.1,0.1), 'C':[0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000]}
svm = SVC(kernel='rbf',decision_function_shape='ovr')
clf = GridSearchCV(svm, parameters, scoring='accuracy', cv=10, n_jobs=-1)
clf.fit(X_train_standardized, y2_train)
#clf.cv_results_
best_svm_Genus_s = clf.best_estimator_
print("(Standardized Features)")
print("Best estimator for Genus:", clf.best_estimator_)
print("With a CV score of ", clf.best_score_)
print("Train score for this clf = ", best_svm_Genus_s.score(X_train_standardized, y2_train))

(Standardized Features)
Best estimator for Genus: SVC(C=10, gamma=0.1)
With a CV score of  0.9872925147527534
Train score for this clf =  1.0


In [48]:
#SVC-Gaussian-Species(y3):standardized
parameters = {'gamma':np.arange(0.1,3.1,0.1), 'C':[0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000]}
svm = SVC(kernel='rbf',decision_function_shape='ovr')
clf = GridSearchCV(svm, parameters, scoring='accuracy', cv=10, n_jobs=-1)
clf.fit(X_train_standardized, y3_train)
#clf.cv_results_
best_svm_Species_s = clf.best_estimator_
print("(Standardized Features)")
print("Best estimator for Species:", clf.best_estimator_)
print("With a CV score of ", clf.best_score_)
print("Train score for this clf = ", best_svm_Species_s.score(X_train_standardized, y3_train))

(Standardized Features)
Best estimator for Species: SVC(C=10, gamma=0.1)
With a CV score of  0.985107213859699
Train score for this clf =  1.0


In [51]:
#exact match and hamming score for combined results: Gaussian + standardized Features
preds_Family_s = best_svm_Family_s.predict(X_test_standardized)
preds_Genus_s = best_svm_Genus_s.predict(X_test_standardized)
preds_Species_s = best_svm_Species_s.predict(X_test_standardized)

print("w Gaussian + standardized Features")
num_exact_match = 0
for i in range(y1_test.shape[0]):
    if((preds_Family_s[i]==y1_test.iloc[i])
       and(preds_Genus_s[i]==y2_test.iloc[i])
       and(preds_Species_s[i]==y3_test.iloc[i])):
        num_exact_match = num_exact_match + 1
exact_match_ratio = num_exact_match/y1_test.shape[0]
print("Exact match ratio = ", exact_match_ratio)

num_hamming_loss = 0
for i in range(y1_test.shape[0]):
    if(preds_Family_s[i]!=y1_test.iloc[i]):
        num_hamming_loss = num_hamming_loss + 1
    if(preds_Genus_s[i]!=y2_test.iloc[i]):
        num_hamming_loss = num_hamming_loss + 1
    if(preds_Species_s[i]!=y3_test.iloc[i]):
        num_hamming_loss = num_hamming_loss + 1
hamming_loss = num_hamming_loss/(y1_test.shape[0]*3)
print("hamming score = ", 1 - hamming_loss)
    

w Gaussian + standardized Features
Exact match ratio =  0.9805465493283928
hamming score =  0.9867222479542999


#### 1(b)iii Repeat 1(b)ii with L1-penalized SVMs. Remember to standardize the attributes. Determine the weight of the SVM penalty using 10 fold cross validation.

In [40]:
#SVC-L1Penalty-Family(y1) & standardized
parameters_l1 = {'C':[0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000, 100000]}
svm_l1 = LinearSVC(penalty='l1', multi_class='ovr', dual=False, max_iter=10000)
clf_l1 = GridSearchCV(svm_l1, parameters_l1, scoring='accuracy', cv=10, n_jobs=-1)
clf_l1.fit(X_train_standardized, y1_train)
#clf_l1.cv_results_
best_svm_Family_l1 = clf_l1.best_estimator_
print("(L1 Penalty)")
print("Best estimator for Family:", clf_l1.best_estimator_)
print("With a CV score of ", clf_l1.best_score_)
print("Train score for this clf = ", best_svm_Family_l1.score(X_train_standardized, y1_train))
print(metrics.classification_report(y1_test, best_svm_Family_l1.predict(X_test_standardized)))

(L1 Penalty)
Best estimator for Family: LinearSVC(C=100, dual=False, max_iter=10000, penalty='l1')
With a CV score of  0.9394407365331817
Train score for this clf =  0.9404289118347895
                 precision    recall  f1-score   support

      Bufonidae       0.00      0.00      0.00        22
  Dendrobatidae       0.90      0.91      0.90       162
        Hylidae       0.91      0.88      0.90       626
Leptodactylidae       0.94      0.97      0.96      1349

       accuracy                           0.93      2159
      macro avg       0.69      0.69      0.69      2159
   weighted avg       0.92      0.93      0.93      2159



/Users/lynz/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/lynz/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/lynz/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [42]:
#SVC-L1Penalty-Genus(y2) & standardized
parameters_l1 = {'C':[0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000, 100000]}
svm_l1 = LinearSVC(penalty='l1', multi_class='ovr', dual=False, max_iter=10000)
clf_l1 = GridSearchCV(svm_l1, parameters_l1, scoring='accuracy', cv=10, n_jobs=-1)
clf_l1.fit(X_train_standardized, y2_train)
#clf_l1.cv_results_
best_svm_Genus_l1 = clf_l1.best_estimator_
print("(L1 Penalty)")
print("Best estimator for Genus:", clf_l1.best_estimator_)
print("With a CV score of ", clf_l1.best_score_)
print("Train score for this clf = ", best_svm_Genus_l1.score(X_train_standardized, y2_train))
print(metrics.classification_report(y2_test, best_svm_Genus_l1.predict(X_test_standardized)))

(L1 Penalty)
Best estimator for Genus: LinearSVC(C=10, dual=False, max_iter=10000, penalty='l1')
With a CV score of  0.9511541859951402
Train score for this clf =  0.9563145353455124
               precision    recall  f1-score   support

    Adenomera       0.96      0.99      0.98      1248
     Ameerega       0.93      0.93      0.93       162
Dendropsophus       0.95      0.65      0.77       108
    Hypsiboas       0.91      0.97      0.94       441
Leptodactylus       0.99      0.91      0.95       101
Osteocephalus       0.88      0.48      0.62        29
     Rhinella       0.92      0.55      0.69        22
       Scinax       0.93      0.90      0.91        48

     accuracy                           0.95      2159
    macro avg       0.93      0.80      0.85      2159
 weighted avg       0.95      0.95      0.94      2159



In [43]:
#SVC-L1Penalty-Species(y3) & standardized
parameters_l1 = {'C':[0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000, 100000]}
svm_l1 = LinearSVC(penalty='l1', multi_class='ovr', dual=False, max_iter=10000)
clf_l1 = GridSearchCV(svm_l1, parameters_l1, scoring='accuracy', cv=10, n_jobs=-1)
clf_l1.fit(X_train_standardized, y3_train)
#clf_l1.cv_results_
best_svm_Species_l1 = clf_l1.best_estimator_
print("(L1 Penalty)")
print("Best estimator for Species:", clf_l1.best_estimator_)
print("With a CV score of ", clf_l1.best_score_)
print("Train score for this clf = ", best_svm_Species_l1.score(X_train_standardized, y3_train))
print(metrics.classification_report(y3_test, best_svm_Species_l1.predict(X_test_standardized)))

(L1 Penalty)
Best estimator for Species: LinearSVC(C=10, dual=False, max_iter=10000, penalty='l1')
With a CV score of  0.9616771592666226
Train score for this clf =  0.9662430500397141
                        precision    recall  f1-score   support

        AdenomeraAndre       0.90      0.95      0.92       208
AdenomeraHylaedactylus       0.99      1.00      0.99      1040
    Ameeregatrivittata       0.91      0.94      0.93       162
            HylaMinuta       0.91      0.68      0.78       108
  HypsiboasCinerascens       0.90      0.93      0.92       136
     HypsiboasCordobae       0.91      0.95      0.93       305
   LeptodactylusFuscus       0.97      0.92      0.94       101
 OsteocephalusOophagus       0.95      0.62      0.75        29
     Rhinellagranulosa       0.95      0.91      0.93        22
           ScinaxRuber       0.94      0.94      0.94        48

              accuracy                           0.95      2159
             macro avg       0.93      0.88  

In [8]:
#exact match and hamming score for combined results: L1 Penalty + standardized Features
preds_Family_l1 = best_svm_Family_l1.predict(X_test_standardized)
preds_Genus_l1 = best_svm_Genus_l1.predict(X_test_standardized)
preds_Species_l1 = best_svm_Species_l1.predict(X_test_standardized)

print("w L1 Penalty + standardized Features")
num_exact_match = 0
for i in range(y1_test.shape[0]):
    if((preds_Family_l1[i]==y1_test.iloc[i])
       and(preds_Genus_l1[i]==y2_test.iloc[i])
       and(preds_Species_l1[i]==y3_test.iloc[i])):
        num_exact_match = num_exact_match + 1
exact_match_ratio = num_exact_match/y1_test.shape[0]
print("Exact match ratio = ", exact_match_ratio)

num_hamming_loss = 0
for i in range(y1_test.shape[0]):
    if(preds_Family_l1[i]!=y1_test.iloc[i]):
        num_hamming_loss = num_hamming_loss + 1
    if(preds_Genus_l1[i]!=y2_test.iloc[i]):
        num_hamming_loss = num_hamming_loss + 1
    if(preds_Species_l1[i]!=y3_test.iloc[i]):
        num_hamming_loss = num_hamming_loss + 1
hamming_loss = num_hamming_loss/(y1_test.shape[0]*3)
print("hamming score = ", 1 - hamming_loss)
    

w L1 Penalty + standardized Features
Exact match ratio =  0.9101435849930524
hamming score =  0.9431835726416551


#### 1(b)iv Repeat 1(b)iii by using SMOTE or any other method you know to remedy class imbalance. Report your conclusions about the classifiers you trained.

In [44]:
#SVC-SMOTE+L1Penalty-Family(y1)
#svm_l1 = LinearSVC(penalty='l1', multi_class='ovr', dual=False, max_iter=10000)
pipeline = imbpipeline(steps = [['smote', SMOTE(random_state=13)],
                                ['classifier', LinearSVC(penalty='l1', multi_class='ovr', dual=False, max_iter=10000)]])
parameters_l1 = {'classifier__C':[0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000]}
clf_smote = GridSearchCV(estimator=pipeline, param_grid=parameters_l1,
                           scoring='accuracy', cv=10, n_jobs=-1)

clf_smote.fit(X_train_standardized, y1_train)
best_svm_Family_smote = clf_smote.best_estimator_
print("(SMOTE + L1 Penalty)")
print("Best estimator for Family:", clf_smote.best_estimator_)
print("With a CV score of ", clf_smote.best_score_)
print("Train score for this clf = ", best_svm_Family_smote.score(X_train_standardized, y1_train))
print(metrics.classification_report(y1_test, best_svm_Family_smote.predict(X_test_standardized)))

(SMOTE + L1 Penalty)
Best estimator for Family: Pipeline(steps=[('smote', SMOTE(random_state=13)),
                ['classifier',
                 LinearSVC(C=10, dual=False, max_iter=10000, penalty='l1')]])
With a CV score of  0.921965035185711
Train score for this clf =  0.9231532962668785
                 precision    recall  f1-score   support

      Bufonidae       0.30      0.95      0.45        22
  Dendrobatidae       0.75      0.97      0.85       162
        Hylidae       0.91      0.84      0.88       626
Leptodactylidae       0.96      0.93      0.95      1349

       accuracy                           0.91      2159
      macro avg       0.73      0.92      0.78      2159
   weighted avg       0.93      0.91      0.91      2159



In [45]:
#SVC-SMOTE+L1Penalty-Genus(y2)
#svm_l1 = LinearSVC(penalty='l1', multi_class='ovr', dual=False, max_iter=10000)
pipeline = imbpipeline(steps = [['smote', SMOTE(random_state=13)],
                                ['classifier', LinearSVC(penalty='l1', multi_class='ovr', dual=False, max_iter=10000)]])
parameters_l1 = {'classifier__C':[0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000]}
clf_smote = GridSearchCV(estimator=pipeline, param_grid=parameters_l1,
                           scoring='accuracy', cv=10, n_jobs=-1)

clf_smote.fit(X_train_standardized, y2_train)
best_svm_Genus_smote = clf_smote.best_estimator_
print("(SMOTE + L1 Penalty)")
print("Best estimator for Genus:", clf_smote.best_estimator_)
print("With a CV score of ", clf_smote.best_score_)
print("Train score for this clf = ", best_svm_Genus_smote.score(X_train_standardized, y2_train))
print(metrics.classification_report(y2_test, best_svm_Genus_smote.predict(X_test_standardized)))

(SMOTE + L1 Penalty)
Best estimator for Genus: Pipeline(steps=[('smote', SMOTE(random_state=13)),
                ['classifier',
                 LinearSVC(C=1, dual=False, max_iter=10000, penalty='l1')]])
With a CV score of  0.9152103253494903
Train score for this clf =  0.9211675933280381
               precision    recall  f1-score   support

    Adenomera       0.99      0.91      0.94      1248
     Ameerega       0.84      0.96      0.90       162
Dendropsophus       0.69      0.81      0.75       108
    Hypsiboas       0.96      0.92      0.94       441
Leptodactylus       0.95      0.97      0.96       101
Osteocephalus       0.42      0.90      0.57        29
     Rhinella       0.38      0.95      0.54        22
       Scinax       0.87      0.96      0.91        48

     accuracy                           0.91      2159
    macro avg       0.76      0.92      0.81      2159
 weighted avg       0.94      0.91      0.92      2159



In [46]:
#SVC-SMOTE+L1Penalty-Species(y3)
#svm_l1 = LinearSVC(penalty='l1', multi_class='ovr', dual=False, max_iter=10000)
pipeline = imbpipeline(steps = [['smote', SMOTE(random_state=13)],
                                ['classifier', LinearSVC(penalty='l1', multi_class='ovr', dual=False, max_iter=10000)]])
parameters_l1 = {'classifier__C':[0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000]}
clf_smote = GridSearchCV(estimator=pipeline, param_grid=parameters_l1,
                           scoring='accuracy', cv=10, n_jobs=-1)

clf_smote.fit(X_train_standardized, y3_train)
best_svm_Species_smote = clf_smote.best_estimator_
print("(SMOTE + L1 Penalty)")
print("Best estimator for Species:", clf_smote.best_estimator_)
print("With a CV score of ", clf_smote.best_score_)
print("Train score for this clf = ", best_svm_Species_smote.score(X_train_standardized, y3_train))
print(metrics.classification_report(y3_test, best_svm_Species_smote.predict(X_test_standardized)))

(SMOTE + L1 Penalty)
Best estimator for Species: Pipeline(steps=[('smote', SMOTE(random_state=13)),
                ['classifier',
                 LinearSVC(C=0.1, dual=False, max_iter=10000, penalty='l1')]])
With a CV score of  0.9571077503234562
Train score for this clf =  0.9616759332803813
                        precision    recall  f1-score   support

        AdenomeraAndre       0.96      0.94      0.95       208
AdenomeraHylaedactylus       0.99      0.99      0.99      1040
    Ameeregatrivittata       0.93      0.94      0.93       162
            HylaMinuta       0.90      0.81      0.85       108
  HypsiboasCinerascens       0.91      0.92      0.92       136
     HypsiboasCordobae       0.95      0.93      0.94       305
   LeptodactylusFuscus       0.94      0.94      0.94       101
 OsteocephalusOophagus       0.75      0.83      0.79        29
     Rhinellagranulosa       0.58      1.00      0.73        22
           ScinaxRuber       0.88      0.96      0.92        48

In [8]:
#exact match and hamming score for combined results: L1 Penalty + SMOTE
preds_Family_smote = best_svm_Family_smote.predict(X_test_standardized)
preds_Genus_smote = best_svm_Genus_smote.predict(X_test_standardized)
preds_Species_smote = best_svm_Species_smote.predict(X_test_standardized)

print("w L1 Penalty + SMOTE")
num_exact_match = 0
for i in range(y1_test.shape[0]):
    if((preds_Family_smote[i]==y1_test.iloc[i])
       and(preds_Genus_smote[i]==y2_test.iloc[i])
       and(preds_Species_smote[i]==y3_test.iloc[i])):
        num_exact_match = num_exact_match + 1
exact_match_ratio = num_exact_match/y1_test.shape[0]
print("Exact match ratio = ", exact_match_ratio)

num_hamming_loss = 0
for i in range(y1_test.shape[0]):
    if(preds_Family_smote[i]!=y1_test.iloc[i]):
        num_hamming_loss = num_hamming_loss + 1
    if(preds_Genus_smote[i]!=y2_test.iloc[i]):
        num_hamming_loss = num_hamming_loss + 1
    if(preds_Species_smote[i]!=y3_test.iloc[i]):
        num_hamming_loss = num_hamming_loss + 1
hamming_loss = num_hamming_loss/(y1_test.shape[0]*3)
print("hamming score = ", 1 - hamming_loss)
    

w L1 Penalty + SMOTE
Exact match ratio =  0.8499305233904585
hamming score =  0.9257372240234677


### conclusion:

Compared to L1-penalty SVM without SMOTE, both exact ratio & hamming score decreased in SMOTE version. 

However, model's ability to classify rare classes (like "Bufonidae" in Family) improved. The above RED warning section appeared because "Bufonidae" is a "label with no predicted samples" in L1-penalty SVM without SMOTE. Conclusions are drew from classification reports above.

## 2. K-Means Clustering on a Multi-Class and Multi-Label Data Set
#### (a) Use k-means clustering on the whole Anuran Calls (MFCCs) Data Set (do not split the data into train and test, as we are not performing supervised learning in this exercise). Choose k ∈ {1, 2, . . . , 50} automatically based on one of the methods provided in the slides (CH or Gap Statistics or scree plots or Silhouettes) or any other method you know.
#### (b) In each cluster, determine which family is the majority by reading the true labels. Repeat for genus and species.
#### (c) Now for each cluster you have a majority label triplet (family, genus, species). Calculate the average Hamming distance, Hamming score, and Hamming loss5 between the true labels and the labels assigned by clusters.

In [35]:
Family = ['Bufonidae', 'Dendrobatidae', 'Hylidae', 'Leptodactylidae']
Genus = ['Adenomera', 'Ameerega', 'Dendropsophus', 'Hypsiboas',
         'Leptodactylus', 'Osteocephalus', 'Rhinella', 'Scinax']
Species = ['AdenomeraAndre', 'AdenomeraHylaedactylus','Ameeregatrivittata', 
           'HylaMinuta', 'HypsiboasCinerascens', 'HypsiboasCordobae', 'ScinaxRuber',
           'LeptodactylusFuscus', 'OsteocephalusOophagus', 'Rhinellagranulosa']

Hamming_distance_list = []

#50 batches
for random_state_index in range(50):
    print("-- batch", random_state_index+1, "--")
    
    #choose k by silhouette_score
    silhouette_score = []
    for k in range(2,51):
        kmeans = KMeans(n_clusters=k, init='random', max_iter=10000, 
                        random_state=random_state_index).fit(X)
        labels = kmeans.labels_
        silhouette_score.append(metrics.silhouette_score(X, labels))
    best_k = silhouette_score.index(max(silhouette_score))+2
    print("max silhouette score = ", max(silhouette_score), "obtained when k = ", best_k)
    
    #fit model wiz optimal k 
    kmeans = KMeans(n_clusters=best_k, init='random', max_iter=10000, random_state=random_state_index).fit(X)
    labels = kmeans.labels_
    groups_Family = [[] for i in range(best_k)]
    groups_Genus = [[] for i in range(best_k)]
    groups_Species = [[] for i in range(best_k)]
    for i in range(y.shape[0]):
        groups_Family[labels[i]].append(y['Family'].iloc[i])
        groups_Genus[labels[i]].append(y['Genus'].iloc[i])
        groups_Species[labels[i]].append(y['Species'].iloc[i])
        
    cnt_Family, cnt_Genus, cnt_Species = [0 for i in range(4)], [0 for i in range(8)], [0 for i in range(10)]
    distance, loss, score = [], [], []
    for i in range(best_k):
        group_length = len(groups_Family[i])
        print("  --cluster", i, "-- size =", group_length,"--")
    
        for j in range(4):
            cnt_Family[j] = groups_Family[i].count(Family[j])
        for j in range(8):
            cnt_Genus[j] = groups_Genus[i].count(Genus[j])
        for j in range(10):
            cnt_Species[j] = groups_Species[i].count(Species[j])
            
        major_Family = Family[cnt_Family.index(max(cnt_Family))]
        major_Genus = Genus[cnt_Genus.index(max(cnt_Genus))]
        major_Species = Species[cnt_Species.index(max(cnt_Species))]
        print("  majority triplet = (", major_Family, ",", major_Genus, ",", major_Species, ")")
        
        hamming_distance = (3 * group_length) -  (max(cnt_Family) + max(cnt_Genus) + max(cnt_Species))
        hamming_loss = hamming_distance / (3 * group_length)
        hamming_score = 1 - hamming_loss
        distance.append(hamming_distance)
        loss.append(hamming_loss)
        score.append(hamming_score)
        
    print("  --average hamming--")
    print("  distance =", sum(distance)/len(distance), "score =", sum(score)/len(score),"loss =", sum(loss)/len(loss))
    Hamming_distance_list.append(sum(distance)/len(distance))

-- batch 1 --
max silhouette score =  0.3787509343305295 obtained when k =  4
  --cluster 0 -- size = 3568 --
  majority triplet = ( Leptodactylidae , Adenomera , AdenomeraHylaedactylus )
  --cluster 1 -- size = 1031 --
  majority triplet = ( Dendrobatidae , Ameerega , Ameeregatrivittata )
  --cluster 2 -- size = 614 --
  majority triplet = ( Hylidae , Hypsiboas , HypsiboasCinerascens )
  --cluster 3 -- size = 1982 --
  majority triplet = ( Hylidae , Hypsiboas , HypsiboasCordobae )
  --average hamming--
  distance = 1200.25 score = 0.7178925049375912 loss = 0.28210749506240884
-- batch 2 --
max silhouette score =  0.3787509343305295 obtained when k =  4
  --cluster 0 -- size = 614 --
  majority triplet = ( Hylidae , Hypsiboas , HypsiboasCinerascens )
  --cluster 1 -- size = 1031 --
  majority triplet = ( Dendrobatidae , Ameerega , Ameeregatrivittata )
  --cluster 2 -- size = 1982 --
  majority triplet = ( Hylidae , Hypsiboas , HypsiboasCordobae )
  --cluster 3 -- size = 3568 --
  major

max silhouette score =  0.3787509343305295 obtained when k =  4
  --cluster 0 -- size = 614 --
  majority triplet = ( Hylidae , Hypsiboas , HypsiboasCinerascens )
  --cluster 1 -- size = 3568 --
  majority triplet = ( Leptodactylidae , Adenomera , AdenomeraHylaedactylus )
  --cluster 2 -- size = 1031 --
  majority triplet = ( Dendrobatidae , Ameerega , Ameeregatrivittata )
  --cluster 3 -- size = 1982 --
  majority triplet = ( Hylidae , Hypsiboas , HypsiboasCordobae )
  --average hamming--
  distance = 1200.25 score = 0.7178925049375912 loss = 0.28210749506240884
-- batch 17 --
max silhouette score =  0.3787509343305295 obtained when k =  4
  --cluster 0 -- size = 1031 --
  majority triplet = ( Dendrobatidae , Ameerega , Ameeregatrivittata )
  --cluster 1 -- size = 614 --
  majority triplet = ( Hylidae , Hypsiboas , HypsiboasCinerascens )
  --cluster 2 -- size = 3568 --
  majority triplet = ( Leptodactylidae , Adenomera , AdenomeraHylaedactylus )
  --cluster 3 -- size = 1982 --
  major

max silhouette score =  0.37866647707550904 obtained when k =  4
  --cluster 0 -- size = 3568 --
  majority triplet = ( Leptodactylidae , Adenomera , AdenomeraHylaedactylus )
  --cluster 1 -- size = 615 --
  majority triplet = ( Hylidae , Hypsiboas , HypsiboasCinerascens )
  --cluster 2 -- size = 1033 --
  majority triplet = ( Dendrobatidae , Ameerega , Ameeregatrivittata )
  --cluster 3 -- size = 1979 --
  majority triplet = ( Hylidae , Hypsiboas , HypsiboasCordobae )
  --average hamming--
  distance = 1198.75 score = 0.7180026236414261 loss = 0.28199737635857397
-- batch 32 --
max silhouette score =  0.38523395202479643 obtained when k =  4
  --cluster 0 -- size = 3560 --
  majority triplet = ( Leptodactylidae , Adenomera , AdenomeraHylaedactylus )
  --cluster 1 -- size = 714 --
  majority triplet = ( Leptodactylidae , Adenomera , AdenomeraAndre )
  --cluster 2 -- size = 1102 --
  majority triplet = ( Hylidae , Hypsiboas , HypsiboasCordobae )
  --cluster 3 -- size = 1819 --
  majorit

max silhouette score =  0.3787509343305295 obtained when k =  4
  --cluster 0 -- size = 3568 --
  majority triplet = ( Leptodactylidae , Adenomera , AdenomeraHylaedactylus )
  --cluster 1 -- size = 614 --
  majority triplet = ( Hylidae , Hypsiboas , HypsiboasCinerascens )
  --cluster 2 -- size = 1982 --
  majority triplet = ( Hylidae , Hypsiboas , HypsiboasCordobae )
  --cluster 3 -- size = 1031 --
  majority triplet = ( Dendrobatidae , Ameerega , Ameeregatrivittata )
  --average hamming--
  distance = 1200.25 score = 0.7178925049375912 loss = 0.28210749506240884
-- batch 47 --
max silhouette score =  0.3787509343305295 obtained when k =  4
  --cluster 0 -- size = 3568 --
  majority triplet = ( Leptodactylidae , Adenomera , AdenomeraHylaedactylus )
  --cluster 1 -- size = 614 --
  majority triplet = ( Hylidae , Hypsiboas , HypsiboasCinerascens )
  --cluster 2 -- size = 1031 --
  majority triplet = ( Dendrobatidae , Ameerega , Ameeregatrivittata )
  --cluster 3 -- size = 1982 --
  major

### Monte-Carlo Simulation: 
 
#### Perform the following procedures 50 times, and report the average and standard deviation of the 50 Hamming Distances that you calculate.

In [38]:
mean = sum(Hamming_distance_list)/50
std = (sum([((x - mean) ** 2) for x in Hamming_distance_list]) / 50) ** 0.5
print("average Hamming distance of 50 batches:", mean)
print("std of Hamming distance:", std)

average Hamming distance of 50 batches: 1230.13
std of Hamming distance: 84.01559736144236


## 3. ISLR 12.6.2

### Suppose that we have four observations, for which we compute a dissimilarity matrix.

#### (a) On the basis of this dissimilarity matrix, sketch the dendrogram that results from hierarchically clustering these four observa- tions using complete linkage. Be sure to indicate on the plot the height at which each fusion occurs, as well as the observations corresponding to each leaf in the dendrogram.

#### (b) Repeat (a), this time using single linkage clustering.

#### (c) Suppose that we cut the dendrogram obtained in (a) such that two clusters result. Which observations are in each cluster?

#### (d) Suppose that we cut the dendrogram obtained in (b) such that two clusters result. Which observations are in each cluster?

#### (e) It is mentioned in the chapter that at each fusion in the den- drogram, the position of the two clusters being fused can be swapped without changing the meaning of the dendrogram. Draw a dendrogram that is equivalent to the dendrogram in (a), for which two or more of the leaves are repositioned, but for which the meaning of the dendrogram is the same.

If not displayed please refer to 'sol' folder.
<img src='../sol/ISLR_12_6_2.jpg'/>